In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt       # чтобы выводить промежуточные фото в jupyter

# Список всех настроечных параметров/констант
WORK_DIR = 'pass_photos'
TEMP_DIR = 'pass_temp'
# TEST_FILE = 'pass_photos/1.jpeg'
IMG_HEIGHT = 1000            # требуемый размер фото для нормализации всех изображений
IMG_WIDTH = 600              # т.к. в задачу входит прочитать только ФИО, обрезаю серию/номер чтобы не усложнять распознавание
INDENT_LEFT = 200            # обрезаем фото т.к. без него получается лучше разделить фото на куски текста
INDENT_BOTTOM = 200          # обрезаем нижние поля
INDENT_TOP = 40              # обрезаем лишнюю часть паспорта снизу

In [2]:
# Функция для получения списка файлов из каталога с фотографиями (как в task_1 и task_2)
def get_files(directory: str) -> list:
    names = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"):
            names.append(os.path.join(directory, filename))

    return names

In [3]:
# Подготовка изображений для распознавания текста
def normalize_image(image: str):    
    img = cv2.imread(image)
    
    # нормализуем фото к нужному размеру
    height, width, channels = img.shape
    resize_scale = IMG_HEIGHT / height       # получаем коэффициент масштабирования изображения
    img_width = int(width * resize_scale)    # высчитываем от этого коэффициента новую ширину
    img = cv2.resize(img, (img_width, IMG_HEIGHT))
    
    # обрезаем паспорт до страницы с фото
    x0 = INDENT_LEFT                            # отступ слева, т.к. корочка и фото нам не важны
    y0 = IMG_HEIGHT // 2 + INDENT_TOP           # обрезка сверху, т.к. верхняя страница с местом выдачи нам не важна 
    x1 = img_width if img_width < IMG_WIDTH else IMG_WIDTH     # обрезаем все лишнее справа, если есть разворот с пропиской
    y1 = IMG_HEIGHT - INDENT_BOTTOM
    img = img[y0:y1, x0:x1]
    
    # обесцвечиваем и пытаемся снизить шум с помощью размытия
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    # преобразуем в ЧБ
    img = cv2.GaussianBlur(img, (5,5), 0)         # коэффициент размытия подобран вручную
    
    
    # одно изображение используем для распознавания блоков текста. очередность преобраозвания найдена методом тыка
    kernel = np.ones((5,5), 'uint8')
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    img_block = cv2.erode(img, kernel, iterations=1)
    #img_block = cv2.dilate(img_block, kernel, iterations=1)
    _, img_block = cv2.threshold(img_block, 0, 255, cv2.THRESH_OTSU, cv2.THRESH_BINARY_INV)
    img_block = cv2.morphologyEx(img_block, cv2.MORPH_OPEN, kernel, iterations=1)
    # img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    
    # второе изображение используем для распознавания букв внутри блоков
    # при коэффициенте 3 - лучше распознается Васлевский, при 5 - Соколов и Юмакаева
    img_symbol = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 2)
    

    return img_block, img_symbol

In [19]:
# Выделяем элементы текста из изображения
def search_blocks(image):
    
    height, width = image.shape
    # получаем контуры больших пятен на изображении, внутри которых спрятан текст
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    output = image.copy()
    
    print(f'Count of Block counoturs: {len(contours)}')
    blocks = []
    for idx, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        print("R", x, y, w, h, hierarchy[0][idx])
        # hierarchy[i][0]: следующий контур текущего уровня
        # hierarchy[i][1]: предыдущий контур текущего уровня
        # hierarchy[i][2]: первый вложенный элемент
        # hierarchy[i][3]: родительский элемент
        # if hierarchy[0][idx][3] == 0:               # если элемент не является самым крупным
        cv2.rectangle(image, (x, y), (x + w, y + h), (70, 0, 0), 1) # для отображаемой картинки
        if 15 < h < height and 15 < w < width:    # игнорируем маленькие блоки, а также блок размером с изображение
            block = image[y:y + h, x:x + w]     # вырезаем найденный блок из изображения
            blocks.append((idx, y, h, x, w, block))  # сохраняем габариты и изображение блока в список блоков
            
    #cv2.imshow("Input", image)
    # cv2.imshow("Output", output)
    #cv2.waitKey(0)
    #print(blocks)
    return blocks
    # return image

In [20]:
# Запускаем цикл по всем фото в рабочей папке
passports = get_files(WORK_DIR)
# col = 5    # количество колонок таблицы
words = []
for i, p in enumerate(passports):
    temp_dir = os.path.join(TEMP_DIR, str(i))
    if not os.path.exists(temp_dir):
        os.mkdir(temp_dir)
        
    print(f'==== Image {i}.jpg =====')
    img_blocks, img_symbols = normalize_image(p)
    
    cv2.imwrite(f'{TEMP_DIR}/{i}_blocs.jpg', img_blocks)
    #cv2.imwrite(f'{TEMP_DIR}/{i}_blocs.jpg', search_blocks(blocks))
    cv2.imwrite(f'{TEMP_DIR}/{i}_symbols.jpg', img_symbols)
    
    words = search_blocks(image=img_blocks)
    print(f'Count of words: {len(words)}')
    
    # получаем все обнаруженные слова из файла, в котором читаются символы
    for word in words:
        y, h, x, w = word[1:5]     # из списка обнаруженных блоков текста забираем координаты этих блоков
        img_word = img_symbols[y:y + h, x:x + w]     # вырезаем слово по его координатам
        cv2.imwrite(os.path.join(temp_dir, f'{word[0]}.jpg'), img_word)
    
    
    #plt.subplot(len(passports)//col+1, col, i+1)    # высчитываем высоту таблицы от количества колонок и общего размера списка
    #plt.imshow(image, 'gray')
    # plt.title(p)
    #plt.xticks([]),plt.yticks([])
#plt.show()

==== Image 0.jpg =====
Count of Block counoturs: 18
R 0 0 400 260 [-1 -1  1 -1]
R 157 223 62 22 [ 2 -1 -1  0]
R 230 219 120 26 [ 3  1 -1  0]
R 44 198 51 13 [ 4  2 -1  0]
R 251 186 76 21 [ 5  3 -1  0]
R 179 186 57 22 [ 6  4 -1  0]
R 45 186 32 10 [ 7  5 -1  0]
R 44 161 21 12 [ 8  6 -1  0]
R 77 154 59 21 [ 9  7 -1  0]
R 221 152 84 22 [10  8 -1  0]
R 308 151 59 23 [11  9 -1  0]
R 146 149 50 26 [12 10 -1  0]
R 48 127 43 11 [13 11 -1  0]
R 225 112 63 22 [14 12 -1  0]
R 44 90 25 12 [15 13 -1  0]
R 199 75 107 23 [16 14 -1  0]
R 45 19 46 12 [17 15 -1  0]
R 211 5 77 21 [-1 16 -1  0]
Count of words: 11
==== Image 1.jpg =====
Count of Block counoturs: 16
R 20 240 5 8 [ 1 -1 -1 -1]
R 0 0 400 260 [-1  0  2 -1]
R 115 188 140 19 [ 3 -1 -1  1]
R 47 184 50 27 [ 4  2 -1  1]
R 47 162 21 12 [ 5  3 -1  1]
R 289 153 42 19 [ 6  4 -1  1]
R 255 153 23 19 [ 7  5 -1  1]
R 221 153 22 19 [ 8  6 -1  1]
R 91 153 22 19 [ 9  7 -1  1]
R 145 151 49 25 [10  8 -1  1]
R 47 129 45 12 [11  9 -1  1]
R 112 119 124 20 [12 10 -1 